<a href="https://colab.research.google.com/github/Sree1259/RAG_Pipeline/blob/main/End_End_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step-by-step Flow

1. Upload a PDF file.
2. Load the document using a document loader.
3. Combine document pages into a single text.
4. Split the text into chunks.
5. Convert chunks into `Document` objects.
6. Generate embeddings and store them in a vector store (FAISS).
7. Perform similarity search and retrieval.
8. Pass retrieved context to an LLM using a prompt.
9. Build a simple chatbot on top of the retrieval pipeline.


## 1. Installing Required Libraries

In [1]:
# Install dependencies: LangChain, Google GenAI, FAISS vector store, and PDF parser
!pip install -q langchain-community langchain-google-genai faiss-cpu pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


##2. File Upload (Google Colab)

In [2]:
from google.colab import files

# Upload PDF file from local machine
uploaded = files.upload()

Saving cric.pdf to cric.pdf


In [4]:
# Extract filename from uploaded files dictionary
filename = list(uploaded.keys())[0]

##3. Document Loaders

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

# Load PDF using PyMuPDF loader
loader = PyMuPDFLoader(filename)

# Extract pages as Document objects
docs = loader.load()

In [6]:
# Display loaded document pages
print(docs)

[Document(metadata={'producer': 'Skia/PDF m137', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/137.0.0.0 Safari/537.36', 'creationdate': '2025-08-01T10:42:48+00:00', 'source': 'cric.pdf', 'file_path': 'cric.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': 'Cricket Laws', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-01T10:42:48+00:00', 'trapped': '', 'modDate': "D:20250801104248+00'00'", 'creationDate': "D:20250801104248+00'00'", 'page': 0}, page_content='Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.\nTwenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by c

##4. Combining Document Content

In [7]:
full_text = ""

# Concatenate all page contents into single text string
for doc in docs:
  full_text += doc.page_content + "\n"

print(full_text)

Cricket Laws
🏏 1. Basics of the Game
Objective: Score more runs than the opponent.
Teams: Two teams of 11 players each.
Innings: Each team gets one or two innings depending on the format.
Formats:
Test Cricket: Up to 5 days, unlimited overs.
One Day International (ODI): 50 overs per side.
Twenty20 (T20): 20 overs per side.
🧢 2. Player Roles
Batsmen: Aim to score runs.
Bowlers: Aim to dismiss batsmen and restrict runs.
Fielders: Support bowlers by catching the ball and stopping runs.
Wicketkeeper: Specialized fielder behind the stumps.
⚖️ 3. Scoring Rules
Run: Completed when batsmen cross and reach opposite creases.
Four (4 runs): Ball hits ground before boundary.
Six (6 runs): Ball crosses boundary without touching the ground.
Extras:
Wide ball: 1 run and extra delivery.
No ball: 1 run and extra delivery.
Cricket Laws
1
Bye: Runs taken without touching the bat.
Leg bye: Runs taken after ball hits the batsman’s body.
🚫 4. Dismissals (Ways to Get Out)
1. Bowled
2. Caught
3. Leg Before Wi

##5. Text Splitters

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split text into chunks with overlap for context preservation
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,  # Maximum characters per chunk
    chunk_overlap = 10  # Overlapping characters between chunks
)

# Split full text into list of text chunks
text_chunks = splitter.split_text(full_text)

print(text_chunks)
print(len(text_chunks))

['Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.', 'Twenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by catching the ball and stopping runs.\nWicketkeeper: Specialized fielder behind the stumps.\n⚖️ 3. Scoring Rules', 'Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1', '1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\

##6. Converting Text Chunks into Document Objects

In [9]:
from langchain_core.documents import Document

# Wrap each text chunk in Document object for vector store compatibility
chunks = [Document(page_content=chunk) for chunk in text_chunks]

print(chunks)

[Document(metadata={}, page_content='Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.'), Document(metadata={}, page_content='Twenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by catching the ball and stopping runs.\nWicketkeeper: Specialized fielder behind the stumps.\n⚖️ 3. Scoring Rules'), Document(metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1'), Document(metadata={}, page_con

##7. Vector Stores

In [10]:
from google.colab import userdata
# Retrieve Gemini API key from Colab secrets
gemini_key = userdata.get('GEMINI')

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize Gemini embedding model with 16-dimensional output
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001",api_key = gemini_key,output_dimensionality=16)

# Create FAISS vector store from document chunks and embeddings
vector_store = FAISS.from_documents(documents=chunks,embedding=embedding_model)



In [14]:
embedding = embedding_model.embed_query(chunks[0].page_content)
print("Embedding length:", len(embedding))
print("Embedding:", embedding)


Embedding length: 16
Embedding: [0.0020087194, -0.007165032, 0.012319044, -0.031602915, 0.01868384, 0.022820605, 0.0133292135, 0.021349585, 0.008754843, 0.021448651, -0.021350807, -0.018449841, -0.007594721, 0.018067593, 0.11131797, 0.014249652]


##8. Similarity Search (Testing Vector Store)



In [15]:
# Test vector store by searching for top 2 similar chunks
results = vector_store.similarity_search("What is leg bye",k=2)

print(results)

[Document(id='650574ad-92ad-4f67-97a9-2f2df1fea2e9', metadata={}, page_content='1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\n1. Bowled\n2. Caught\n3. Leg Before Wicket (LBW)\n4. Run Out\n5. Stumped\n6. Hit Wicket\n7. Handled the Ball (obsolete in some formats)\n8. Obstructing the Field'), Document(id='c4ddf7a8-5e30-474e-8130-ce91fb611d93', metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1')]


##9. Retriever

In [16]:
# Convert vector store to retriever interface for RAG pipeline
retriever = vector_store.as_retriever(
    search_type="similarity",  # Use cosine similarity search
    search_kwargs={"k":3}  # Return top 3 most relevant chunks
)

In [17]:
# Test retriever with sample query
results = retriever.invoke("What is leg bye")

print(results)

[Document(id='650574ad-92ad-4f67-97a9-2f2df1fea2e9', metadata={}, page_content='1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\n1. Bowled\n2. Caught\n3. Leg Before Wicket (LBW)\n4. Run Out\n5. Stumped\n6. Hit Wicket\n7. Handled the Ball (obsolete in some formats)\n8. Obstructing the Field'), Document(id='c4ddf7a8-5e30-474e-8130-ce91fb611d93', metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1'), Document(id='11c9254a-8952-4e78-8a2a-46ee67a3d01c', metadata={}, page_content='ODI: First 10 overs - only 2 fielders outside the 30-yard circle.\nT20: First 6 overs - same restriction.\n⏱️ 8. Timing and Match Duration\nTest: 5 days, 90 overs/day.\nODI: ~8 h

##10. Preparing Context for the LLM

In [18]:
def get_context_string(documents):
  """Concatenate retrieved document chunks into single context string."""
  context_text= ""
  for doc in documents:
    context_text+= doc.page_content + "\n"

  return context_text

In [19]:
# Convert retrieved documents to context string
context= get_context_string(results)

print(context)

1
Bye: Runs taken without touching the bat.
Leg bye: Runs taken after ball hits the batsman’s body.
🚫 4. Dismissals (Ways to Get Out)
1. Bowled
2. Caught
3. Leg Before Wicket (LBW)
4. Run Out
5. Stumped
6. Hit Wicket
7. Handled the Ball (obsolete in some formats)
8. Obstructing the Field
Run: Completed when batsmen cross and reach opposite creases.
Four (4 runs): Ball hits ground before boundary.
Six (6 runs): Ball crosses boundary without touching the ground.
Extras:
Wide ball: 1 run and extra delivery.
No ball: 1 run and extra delivery.
Cricket Laws
1
ODI: First 10 overs - only 2 fielders outside the 30-yard circle.
T20: First 6 overs - same restriction.
⏱️ 8. Timing and Match Duration
Test: 5 days, 90 overs/day.
ODI: ~8 hours with innings break.
T20: ~3 hours total.
🧾 9. Umpiring and Decisions



##11. LLM and Prompt Template

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

# Initialize Gemini LLM
llm  = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key = gemini_key)

# Define RAG prompt template with context and question placeholders
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful cricket expert.
Answer ONLY using the following context from the uploaded PDF.
If the context is insufficient, just say "I don't know".

Context:
{context}

Question:
{question}
"""
)

In [21]:
chain = prompt | llm

In [22]:
query = "Who is Salman Khan"

retrieved_docs = retriever.invoke(query)


final_retrieved_docs = get_context_string(retrieved_docs)

print(final_retrieved_docs)


response = chain.invoke({
    "question": query,
    "context": final_retrieved_docs
})


print(response.content)

Twenty20 (T20): 20 overs per side.
🧢 2. Player Roles
Batsmen: Aim to score runs.
Bowlers: Aim to dismiss batsmen and restrict runs.
Fielders: Support bowlers by catching the ball and stopping runs.
Wicketkeeper: Specialized fielder behind the stumps.
⚖️ 3. Scoring Rules
1
Bye: Runs taken without touching the bat.
Leg bye: Runs taken after ball hits the batsman’s body.
🚫 4. Dismissals (Ways to Get Out)
1. Bowled
2. Caught
3. Leg Before Wicket (LBW)
4. Run Out
5. Stumped
6. Hit Wicket
7. Handled the Ball (obsolete in some formats)
8. Obstructing the Field
Cricket Laws
🏏 1. Basics of the Game
Objective: Score more runs than the opponent.
Teams: Two teams of 11 players each.
Innings: Each team gets one or two innings depending on the format.
Formats:
Test Cricket: Up to 5 days, unlimited overs.
One Day International (ODI): 50 overs per side.

I don't know.
